In [ ]:
from animationpipeline import AnimateInferPipeline, use_raw_vid, use_vid_key_points
from src.utils.util import get_fps, read_frames, save_videos_from_pil, save_videos_grid
from src.dwpose import DWposeDetector
from PIL import Image
import matplotlib.pyplot as plt
from tools.openpose_rescaler import draw, rescale_skeleton
from tools import openpose_rescaler
import importlib
from IPython.display import Video
import json
import cv2
import numpy as np
import copy
import torch

In [ ]:
detector = DWposeDetector().to('cuda')

In [ ]:
animator = AnimateInferPipeline(
    base_model_path='./pretrained_weights/stable-diffusion-v1-5',
    reference_unet_path="/cephfs/SZ-AI/usr/liuchenyu/HaiLook/Moore-AnimateAnyone/pretrained_weights/opensource_stage1/reference_unet-0.pth",
    denoising_unet_path="/cephfs/SZ-AI/usr/liuchenyu/HaiLook/Moore-AnimateAnyone/pretrained_weights/opensource_stage1/denoising_unet-0.pth", 
    vae_path='./pretrained_weights/sd-vae-ft-mse',
    pose_guider_path="/cephfs/SZ-AI/usr/liuchenyu/HaiLook/Moore-AnimateAnyone/pretrained_weights/opensource_stage1/pose_guider-0.pth",
    motion_module_path="/cephfs/SZ-AI/usr/liuchenyu/HaiLook/Moore-AnimateAnyone/stage2_out/mmV3/motion_module-3585.pth",
    image_encoder_path='./pretrained_weights/sd-image-variations-diffusers/image_encoder',
    infer_config_path="/cephfs/SZ-AI/usr/liuchenyu/HaiLook/Moore-AnimateAnyone/configs/inference/inference_v2.yaml",
    # lora_path="/cephfs/SZ-AI/usr/liuchenyu/HaiLook/Moore-AnimateAnyone/stage2_out/mmV3_lora_full_data_5e-5/lora-2604.pth",
    # motion_module_path='/cephfs/SZ-AI/usr/liuchenyu/HaiLook/Moore-AnimateAnyone/pretrained_weights/v3_sd15_mm.ckpt'
)

In [ ]:
torch.cuda.empty_cache()

In [ ]:
image_dir = "/cephfs/SZ-AI/usr/liuchenyu/HaiLook/Moore-AnimateAnyone/assets/ref_image/ComfyUI_temp_yeppq_00019_.png"

H = 512
W = 512

ref_image = cv2.resize(np.array(Image.open(image_dir)), (W,H))
ref_pose, score, ref_pose_keypoints = detector(ref_image, output_type='key_points')

fig, axs = plt.subplots(1, 2, figsize=(15, 8))
axs[0].imshow(ref_image)
axs[1].imshow(ref_pose)
axs[0].axis('off')
axs[1].axis('off')

In [ ]:
use_raw = 1

raw_video_dir = '/cephfs/SZ-AI/usr/liuchenyu/HaiLook/Moore-AnimateAnyone/assets/video_clips/tiktok_vlip_0.mp4'

pose_vid_dir = '/cephfs/SZ-AI/usr/liuchenyu/HaiLook/vid_dataset_hdtf_tiktok/videos_dwpose/00054.mp4'

if use_raw:
    pil_vid, pose_vid_dir = use_raw_vid(
        raw_video_dir,
        W,
        H,
        detector,
        'lean_on_arm',
        # ref_pose_keypoints,
        start_frame=547,
        end_frame=582
    )
    
else:
    pil_vid = read_frames(pose_vid_dir)
    print("Loaded dwpose frames, total frames =", len(pil_vid))
    
for i in range(len(pil_vid)):
        pil_vid[i] = Image.fromarray(cv2.resize(np.array(pil_vid[i]), (W,H), interpolation=cv2.INTER_LINEAR))
        

In [ ]:
res_vid = animator.animate(
    ref_image=ref_image, 
    pose_list=pil_vid, 
    width=W, 
    height=H,
    video_length=35,
    num_inference_steps=20,
    cfg=3.5,
    seed=42,
    context_frames=16
)

torch.cuda.empty_cache()

In [ ]:
save_video_name = 'leaning_v3'

save_video_dir = '/cephfs/SZ-AI/usr/liuchenyu/HaiLook/Moore-AnimateAnyone/output/inference/' + save_video_name + '.mp4'

save_videos_grid(res_vid, save_video_dir,n_rows=1,fps=30)

Video(save_video_dir)